In [1]:
import spacy
import pdfminer
import re
import os
import logging
import pandas as pd

In [2]:
!ls

__pycache__          output               resume_parsing.ipynb
csv                  pdf2txt.py           resumes


In [3]:
import pdf2txt

In [4]:
def convert_pdf(f):

    output_filename = os.path.basename(os.path.splitext(f)[0]) + '.txt'
    #output_filepath = os.path.join('..', 'data', 'output', output_filename)
    output_filepath = os.path.join('output/txt/', output_filename)
    #logging.info('Writing text from {} to {}'.format(f, output_filepath))
    pdf2txt.main(args=[f, '--outfile', output_filepath])
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

In [5]:
os.path.splitext("Sally CrookCV.pdf")[0]

'Sally CrookCV'

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
#phone_num credit https://stackoverflow.com/a/3868861

In [8]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []}
names = []
phones = []
emails = []
skills = []

In [9]:
def parse_content(text):
    skillset = re.compile('python|java')
    phone_num = re.compile('(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    doc = nlp(text)
    #for entity in doc.ents:
    #    print(entity.text, entity.label_)
    #result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []}

    name = [entity.text for entity in doc.ents if entity.label_ is 'PERSON'][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num,text.lower()))
    skills_list = re.findall(skillset,text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [10]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join('resumes/',file))
        parse_content(txt)

Reading.....Sally CrookCV.pdf
output/txt/Sally CrookCV.txt saved successfully!!!
Sally Crook
crooksally@memail.com
Extraction completed successfully!!!
Reading.....Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved successfully!!!
Alisson Parker-Wright
alli1414parks@mail.com
Extraction completed successfully!!!
Reading.....John DominicCV.pdf
output/txt/John DominicCV.txt saved successfully!!!
John Dominic 


johndominic@mail.com
Extraction completed successfully!!!


In [11]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills
print(result_dict)

{'name': ['Sally Crook', 'Alisson Parker-Wright', 'John Dominic \n\n'], 'phone': ["['6592251400']", "['8569878511']", "['7877756411']"], 'email': [crooksally@memail.com, alli1414parks@mail.com, johndominic@mail.com], 'skills': ['set()', "{'java', 'python'}", "{'java', 'python'}"]}


In [12]:
pd.DataFrame(result_dict)

,name,phone,email,skills
0,Sally Crook,['6592251400'],crooksally@memail.com,set()
1,Alisson Parker-Wright,['8569878511'],alli1414parks@mail.com,"{'java', 'python'}"
2,John Dominic \n\n,['7877756411'],johndominic@mail.com,"{'java', 'python'}"
